**Work for mc problems, but output weird predictions for other type.**

In [1]:
import os
import json
import pickle
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
autocast_questions = json.load(open('/content/gdrive/My Drive/autocast_questions.json')) # from the Autocast dataset
test_questions = json.load(open('/content/gdrive/My Drive/autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]

## Create baseline models outputting random answers

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00


In [ ]:

# Remove examples with Ids in the test set from the training set
filtered_train_data = [example for example in autocast_questions if example["id"] not in test_ids and example["answer"] is not None]

# Save the filtered training set to a new file
with open("filtered_train_data.json", "w") as f:
    json.dump(filtered_train_data, f)

In [34]:

from transformers import AutoTokenizer, AutoModelWithLMHead, T5ForConditionalGeneration, Trainer, TrainingArguments
import json

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-qasc")
model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-qasc")
# Load the autocast dataset


f = open('/content/gdrive/My Drive/filtered_train_data.json')
json_obj = json.load(f)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [60]:
import pandas as pd
print(len(json_obj))
print(len(json_obj[0]))
train_dataset = pd.DataFrame(json_obj)
column_names = list(train_dataset.keys())
print(column_names)

2797
14
['question', 'id', 'background', 'publish_time', 'close_time', 'tags', 'source_links', 'prediction_count', 'forecaster_count', 'answer', 'choices', 'status', 'qtype', 'crowd']


In [61]:
print(train_dataset['qtype'])

0        mc
1        mc
2       t/f
3       t/f
4       t/f
       ... 
2792    num
2793    num
2794    num
2795    num
2796    num
Name: qtype, Length: 2797, dtype: object


In [66]:
t1 = train_dataset.loc[train_dataset['id'] == 'M402'].index[0]
a1 = train_dataset.loc[t1, 'answer']
q1 = train_dataset.loc[t1, 'question']
b1 = train_dataset.loc[t1, 'background']
print(a1)
print(q1)
print(b1)
print(get_response(q1, b1))

0.42049
How many Federally Insured Commercial Banking Institutions will there be at the end of 2017?
The U.S. has seen a continued consolidation of the Banking Industry over the past 70+ Years. According to the FDIC, there were over 14,000 FDIC Insured Commercial banks in 1934. Over the next 50 years, the Country maintained similar levels of Commercial Banks (i.e., over 14,000 FDIC Insured Commercial banks in 1984). However, over the past 30+ years, this number has shrunk rapidly through bank mergers and failures. As of 3rd quarter of 2016, year end, there are just 5,170 banks in the United States; the lowest year end total on record. Conversely, banks balance sheets have been growing rapidly over the same time horizon. According to the same FDIC report, domestic deposits at these banks as of Q3 2016 are at the highest levels ever, at over $10.5 trillion. The reasoning for this consolidation are wide ranging, including impacts from the Great Recession (and other past recessions or fina

In [62]:
train_dataset.head()

,question,id,background,publish_time,close_time,tags,source_links,prediction_count,forecaster_count,answer,choices,status,qtype,crowd
0,What will the end-of-day closing value for the...,G1,Outcome will be determined by the end-of-day c...,2015-09-01 13:49:29.860000+00:00,2016-01-01 17:00:01+00:00,"[Finance, Economic Indicators]",[http://ftalphaville.ft.com/2015/08/17/2137329...,1549,385,D,"[Less than 6.30, Between 6.30 and 6.35, inclus...",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
1,How many seats will the Justice and Developmen...,G2,The Justice and Development Party (AKP) failed...,2015-09-01 13:54:25.050000+00:00,2015-11-01 22:00:20+00:00,"[Elections and Referenda, Non-US Politics]",[http://www.al-monitor.com/pulse/originals/201...,567,194,A,"[A majority, A plurality, Not a plurality]",Resolved,mc,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
2,Will there be an initial public offering on ei...,G4,China suspended initial public offerings (IPOs...,2015-09-01 13:58:30.138000+00:00,2015-11-30 14:00:15+00:00,[Finance],[http://atimes.com/2015/11/china-will-allow-su...,545,148,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
3,Will the Export-Import Bank of the United Stat...,G5,The Export-Import Bank's authorization expired...,2015-09-01 14:02:21.242000+00:00,2015-12-04 14:00:25+00:00,"[Economic Policy, US Politics, US Policy]",[http://thehill.com/policy/finance/260118-week...,1000,379,yes,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."
4,Will a trilateral meeting take place between C...,G6,"A trilateral meeting of leaders from China, Ja...",2015-09-01 14:04:41.470000+00:00,2015-12-31 23:00:11+00:00,[Foreign Policy],"[https://en.wikipedia.org/wiki/Li_Keqiang, htt...",946,385,no,"[yes, no]",Resolved,t/f,"[{'timestamp': '2015-09-01 00:00:00+00:00', 'f..."


In [22]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

def get_response(question, context, max_length=64):
  input_text = 'question: %s  context: %s' % (question, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return cleanhtml(tokenizer.decode(output[0]))

print(get_response("Where is the capital of China?", "China is in east Asian"))

 Beijing


In [58]:
idx = train_dataset.loc[train_dataset['id'] == 'G1'].index[0]
 
# Extract the context of the row with the given index
answer = train_dataset.loc[idx, 'answer']
choice = train_dataset.loc[idx, 'choices']
# Print the context
print(answer)
opt = ord(answer) - ord('A')
ranswer = choice[opt]
print(ranswer)

D
6.40 or more


In [54]:
idx = train_dataset.loc[train_dataset['id'] == 'G4'].index[0]
question = train_dataset.loc[idx, 'question']
# Extract the context of the row with the given index
answer = train_dataset.loc[idx, 'answer']
choice = train_dataset.loc[idx, 'choices']
ground = train_dataset.loc[idx, 'background']
ground = ground[:64]
print(ground)
# Print the context
print(answer)
print(choice)
print(question)
print(ground)
pred = get_response(train_dataset.loc[idx, 'question'], '')
print(pred)

China suspended initial public offerings (IPOs) in early July (h
yes
['yes', 'no']
Will there be an initial public offering on either the Shanghai Stock Exchange or the Shenzhen Stock Exchange before 1 January 2016?
China suspended initial public offerings (IPOs) in early July (h
[PAD] [unused3] [unused5] [unused3] [unused32] context : [SEP] [unused3] [unused7] [unused3] [unused5] [unused3] [unused5] [unused3] [unused32] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5] [unused3] [unused5]


In [55]:
ts = get_response('Will Iran release Jason Rezaian before 31 October 2016?', ' ')
print(ts)

[PAD] [unused3] context : [SEP] : [SEP] [unused3] [SEP] [unused3] context : [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3] [SEP] [unused3]


In [ ]:
print(get_response())

In [23]:
pred = get_response(train_dataset.loc[idx, 'question'], train_dataset.loc[idx, 'background'])

In [26]:
print(pred)

 usdcny


In [30]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object to convert the sentences to vectors of word counts
vectorizer = CountVectorizer().fit_transform([ranswer, pred])

# Calculate the cosine similarity between the two vectors
cosine_sim = cosine_similarity(vectorizer[0], vectorizer[1])[0][0]

# Print the cosine similarity
print("Cosine similarity:", cosine_sim)


Cosine similarity: 0.0


In [36]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from transformers import AutoTokenizer, AutoModel

tokenizer_acc = AutoTokenizer.from_pretrained("distilbert-base-cased")
model_acc = AutoModel.from_pretrained("distilbert-base-cased")


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:

from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object to convert the sentences to vectors of word counts
vectorizer = CountVectorizer().fit_transform([ranswer, pred])

# Calculate the cosine similarity between the two vectors
cosine_sim = cosine_similarity(vectorizer[0], vectorizer[1])[0][0]

# Print the cosine similarity
print("Cosine similarity:", cosine_sim)


Cosine similarity: 0.0


In [44]:
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Tokenize and encode the sentences
real_answer_tokens = tokenizer.encode(ranswer, return_tensors="pt")
prediction_tokens = tokenizer.encode(pred, return_tensors="pt")

# Generate fixed-length vector representations of the sentences
with torch.no_grad():
    real_answer_vec = model_acc(real_answer_tokens)[0][:,0,:]
    prediction_vec = model_acc(prediction_tokens)[0][:,0,:]

# Compute the cosine similarity between the two sentence vectors
similarity = cosine_similarity(real_answer_vec, prediction_vec)

print(f"Cosine similarity: {similarity[0][0]:.2f}")


Cosine similarity: 0.96


## Get performance on the Autocast train set

Note that the Autocast dataset contains questions in the competition test set. Those should not be used.

In [2]:
def brier_score(probabilities, answer_probabilities):
    return ((probabilities - answer_probabilities) ** 2).sum() / 2

In [ ]:
preds = []
answers = []
qtypes = []
for question in autocast_questions:
    if question['id'] in test_ids: # skipping questions in the competition test set
        continue
    if question['answer'] is None: # skipping questions without answer
        continue
    preds.append(calibrated_random_baseline_model(question))
    if question['qtype'] == 't/f':
        ans_idx = 0 if question['answer'] == 'no' else 1
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('t/f')
    elif question['qtype'] == 'mc':
        ans_idx = ord(question['answer']) - ord('A')
        ans = np.zeros(len(question['choices']))
        ans[ans_idx] = 1
        qtypes.append('mc')
    elif question['qtype'] == 'num':
        ans = float(question['answer'])
        qtypes.append('num')
    answers.append(ans)

## Evaluate the model

In [ ]:
tf_results, mc_results, num_results = [],[],[]
for p, a, qtype in zip(preds, answers, qtypes):
    if qtype == 't/f':
        tf_results.append(brier_score(p, a))
    elif qtype == 'mc':
        mc_results.append(brier_score(p, a))
    else:
        num_results.append(np.abs(p - a))

print(f"T/F: {np.mean(tf_results)*100:.2f}, MCQ: {np.mean(mc_results)*100:.2f}, NUM: {np.mean(num_results)*100:.2f}")
print(f"Combined Metric: {(np.mean(tf_results) + np.mean(mc_results) + np.mean(num_results))*100:.2f}")

T/F: 25.00, MCQ: 38.05, NUM: 22.63
Combined Metric: 85.67


## Make predictions on test set

In [ ]:
preds = []
for question in test_questions:
    preds.append(calibrated_random_baseline_model(question))

In [ ]:
if not os.path.exists('submission'):
    os.makedirs('submission')

with open(os.path.join('submission', 'predictions.pkl'), 'wb') as f:
    pickle.dump(preds, f, protocol=2)

!cd submission && zip ../submission.zip ./* && cd ..

updating: predictions.pkl (deflated 79%)


In [ ]:
!ls

autocast_competition_test_set.json submission
autocast_questions.json            submission.zip
example_submission.ipynb
